Cedric Keller***REMOVED*** & Alexander Zank***REMOVED***
# Assignment 5

This assignment is about visible light communication (VLC). LEDs act as transceivers and communicate using protocols that ensure no difference to "steady" light is visible to humans.

## Step 2

